Table vi_01_q: Entry clearance visa applications and resolution by category

In [1]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='vi_01_q')

In [3]:
sheet = pd.DataFrame(data.get_array())
sheet.rename(columns=sheet.iloc[3], inplace=True)
sheet.drop([0,1,2,3], inplace=True)
sheet.drop(sheet.index[sheet['Quarter'] == ''], inplace=True)
sheet

,Quarter,Type,Broad category,Applicant type,Category,Applications,Resolved,Granted,%,Refused,%,Withdrawn,Lapsed
4,2005 Q1,*Total,*Total,*Total,*Total,549934,544208,428907,0.788131,108812,0.199946,4888,1601
5,2005 Q1,*Total (excl. Visitors and transit),*Total,*Total,*Total (excluding visitors and transit),158424,157409,121258,0.770337,33704,0.214117,2111,336
6,2005 Q1,Work,*Total,*Total,*Total,72805,70631,60550,0.857272,8745,0.123812,1220,116
7,2005 Q1,Study,*Total,*Total,*Total (including Short-term study),47737,49921,29018,0.581278,20236,0.40536,527,140
8,2005 Q1,Study,*Total,*Total,*Total (excluding Short-term study),44870,47049,26335,0.559736,20068,0.426534,517,129
9,2005 Q1,Family,*Total,*Total,*Total,19917,19415,15943,0.821169,3258,0.167808,181,33
10,2005 Q1,Dep. joining or accompanying,*Total,*Total,*Total,8209,8052,6850,0.85072,1105,0.137233,79,18
11,2005 Q1,Other,*Total,*Total,*Total,401266,396189,316546,0.798977,75468,0.190485,2881,1294
12,2005 Q1,Work,Tier 1 & pre-PBS equivalent,Main applicant,*Total,1814,3218,1430,0.444375,1294,0.402113,490,4
13,2005 Q1,Work,Tier 1 & pre-PBS equivalent,Dependant,*Total,59,0,0,0,0,0,0,0


Let's consider the columns up to 'Category' to be the main dimensions, ignore the derived % calculations and collapse down the various types of applications under a 'Resolution' column:

In [46]:
tidy = pd.wide_to_long(
    sheet.drop(columns=['%']).rename(columns={'Applications': 'Applications All',
                    'Resolved': 'Applications Resolved',
                    'Granted': 'Applications Granted',
                    'Refused': 'Applications Refused',
                    'Withdrawn': 'Applications Withdrawn',
                    'Lapsed': 'Applications Lapsed'}),
                stubnames='Applications',
                i=['Quarter', 'Type', 'Broad category', 'Applicant type', 'Category'],
                sep=' ',
                suffix='.*',
                j='Resolution').reset_index()
tidy

,Quarter,Type,Broad category,Applicant type,Category,Resolution,Applications
0,2005 Q1,*Total,*Total,*Total,*Total,All,549934
1,2005 Q1,*Total,*Total,*Total,*Total,Resolved,544208
2,2005 Q1,*Total,*Total,*Total,*Total,Granted,428907
3,2005 Q1,*Total,*Total,*Total,*Total,Refused,108812
4,2005 Q1,*Total,*Total,*Total,*Total,Withdrawn,4888
5,2005 Q1,*Total,*Total,*Total,*Total,Lapsed,1601
6,2005 Q1,*Total (excl. Visitors and transit),*Total,*Total,*Total (excluding visitors and transit),All,158424
7,2005 Q1,*Total (excl. Visitors and transit),*Total,*Total,*Total (excluding visitors and transit),Resolved,157409
8,2005 Q1,*Total (excl. Visitors and transit),*Total,*Total,*Total (excluding visitors and transit),Granted,121258
9,2005 Q1,*Total (excl. Visitors and transit),*Total,*Total,*Total (excluding visitors and transit),Refused,33704


Consider `*Total` to be `All` and rename `Applications` to `Value`

In [47]:
tidy.replace(r'^\*Total', 'All', regex=True, inplace=True)
tidy.rename(columns={'Applications': 'Value'}, inplace=True)
tidy

,Quarter,Type,Broad category,Applicant type,Category,Resolution,Value
0,2005 Q1,All,All,All,All,All,549934
1,2005 Q1,All,All,All,All,Resolved,544208
2,2005 Q1,All,All,All,All,Granted,428907
3,2005 Q1,All,All,All,All,Refused,108812
4,2005 Q1,All,All,All,All,Withdrawn,4888
5,2005 Q1,All,All,All,All,Lapsed,1601
6,2005 Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),All,158424
7,2005 Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Resolved,157409
8,2005 Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Granted,121258
9,2005 Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Refused,33704


Turn the `Quarter` into a standard `Period`, e.g. quarter/2005-Q1

In [48]:
tidy.replace(
    {'Quarter': r'([0-9]{4}) (Q[1-4])'},
    {'Quarter': r'quarter/\1-\2'}, regex=True, inplace=True)
tidy.rename(
    columns={'Quarter': 'Period'}, inplace=True)
tidy

,Period,Type,Broad category,Applicant type,Category,Resolution,Value
0,quarter/2005-Q1,All,All,All,All,All,549934
1,quarter/2005-Q1,All,All,All,All,Resolved,544208
2,quarter/2005-Q1,All,All,All,All,Granted,428907
3,quarter/2005-Q1,All,All,All,All,Refused,108812
4,quarter/2005-Q1,All,All,All,All,Withdrawn,4888
5,quarter/2005-Q1,All,All,All,All,Lapsed,1601
6,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),All,158424
7,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Resolved,157409
8,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Granted,121258
9,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Refused,33704


Todo: data markers, `z` means `not applicable` and `:` means `not available`

In [49]:
import numpy as np
tidy.drop(tidy.index[~tidy['Value'].map(np.isreal)], inplace=True)
tidy['Value'] = tidy['Value'].astype(int)
tidy

,Period,Type,Broad category,Applicant type,Category,Resolution,Value
0,quarter/2005-Q1,All,All,All,All,All,549934
1,quarter/2005-Q1,All,All,All,All,Resolved,544208
2,quarter/2005-Q1,All,All,All,All,Granted,428907
3,quarter/2005-Q1,All,All,All,All,Refused,108812
4,quarter/2005-Q1,All,All,All,All,Withdrawn,4888
5,quarter/2005-Q1,All,All,All,All,Lapsed,1601
6,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),All,158424
7,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Resolved,157409
8,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Granted,121258
9,quarter/2005-Q1,All (excl. Visitors and transit),All,All,All (excluding visitors and transit),Refused,33704
